<a href="https://colab.research.google.com/github/Dattu219/SPARQL-Bot/blob/main/SPARQL_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Chatbot for natural language understanding over linked data***




---


With the rapid progress of the semantic web, a huge amount of structured data 
has become available on the web in the form of knowledge bases (KBs). Making these data accessible and useful for end-users is one of the main objectives of chatbots over linked data. Building a chatbot over linked data raises different challenges, including user queries understanding, multiple knowledge base support, and multilingual aspect. To address these challenges, we first design and develop an architecture to provide an interactive user interface. Secondly, we propose a machine learning approach based on intent classification and natural language understanding to understand user intents and generate SPARQL queries.The system can be extended with a new domain on-demand, flexible, multiple knowledge base, multilingual, and allows intuitive creation and execution of different tasks for an extensive range of topics. Furthermore, evaluation and application cases in the chatbot are provided to show how it facilitates interactive semantic data towards different real application scenarios and showcase the proposed approach for a knowledge graph and data-driven chatbot.

---

# Requirements

Before mounting google drive you need to put following directories in Colab Notebooks folder in google drive.

1. data
2. static
3. templates

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!pip install sparqlwrapper
!pip install vocabulary

In [ ]:
!pip install chatterbot
!pip install chatterbot-corpus

In [ ]:
!pip install flask-ngrok

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Intent Classification

In [ ]:
import spacy
import json
import io
import pickle
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Files to be used
INDEX_TO_LABEL_FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/data/index_to_label.json"
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/data/LabelledData.txt"
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/data/intent_classifier.p"

In [ ]:
spacy_model = spacy.load('en_core_web_sm')

In [ ]:
class DataHelper:
    '''
        DataHelper class is used to take text file containing intents, labels as input
        and convert them to required json labels to be used for intent classification.
    '''

    index_to_label_dict = None

    @classmethod
    def get_index_label_dict(self):
        '''
            get_index_label_dict function is used to create a dictionary if not already
            present and return it.
        '''
        if self.index_to_label_dict is None:
            try:
                with open(INDEX_TO_LABEL_FILE_PATH) as json_data:
                    self.index_to_label_dict = json.load(json_data)
            except:
                print("Unable to read index to index to label dict")
        
        return self.index_to_label_dict

    @classmethod
    def get_data_and_labels(self):
        '''
            get_data_and_labels function is used to extract intents and their labels from
            text file.
        '''

        with io.open(DATA_PATH,"r",encoding="utf-8") as f:
            x_data = []
            y_data = []
            for line in f:
                if(line.__contains__(",,,") and len(line.split(",,,")) == 2):
                    line_splitted = line.split(",,,")
                    x = line_splitted[0].strip()
                    y = line_splitted[1].strip()
                    x_lemmatized = DataHelper.lemmatize(x)
                    x_data.append(np.array(spacy_model(x_lemmatized).vector))
                    y_data.append(y)
        
        return x_data,y_data

    @classmethod
    def lemmatize(self,text):
        '''
            lemmatize function is used to lemmatize the given sentence.
        '''

        sent = []
        doc = spacy_model(text)
        for word in doc:
            sent.append(word.lemma_)

        return " ".join(sent)

    @classmethod
    def save_index_to_label_mapping(self,label_indexes,le):
        '''
            save_index_to_label_mapping function is used to create a json file containing
            labels.
        '''
        
        index_to_label_dict = {}
        label_indexes_list = list(set(label_indexes))
        labels_list = le.inverse_transform(label_indexes_list)
        for i in range(len(labels_list)):
            index_to_label_dict[str(label_indexes_list[i])] = labels_list[i]

        with open(INDEX_TO_LABEL_FILE_PATH, 'w') as outfile:
            json.dump(index_to_label_dict, outfile)

In [ ]:
class IntentClassifier:
    '''
        IntentClassifier class is used to train the model using SVC and predict the
        intents for the given input sentences.
    '''

    classifier = None

    @classmethod
    def get_svm_config(self):
        '''
            get_svm_config function is used to set GridSearch configurations.
        '''

        config = {
            "kernel": "linear",
            "C": [1,2,5,10,20,100]
        }
        return config

    @classmethod
    def train(self):
        '''
            train function is used to train the model using SVC.
        '''

        X, labels = DataHelper.get_data_and_labels()
        X, labels = shuffle(X,labels)
        le = LabelEncoder()
        y = le.fit_transform(labels)

        DataHelper.save_index_to_label_mapping(y,le)

        sklearn_config = IntentClassifier.get_svm_config()
        C = sklearn_config.get("C")
        kernel = sklearn_config.get("kernel")

        tuned_parameters = [{"C": C, "kernel": [str(kernel)]}]

        clf = GridSearchCV(SVC(probability=True, class_weight='balanced',decision_function_shape='ovr'),
                                param_grid=tuned_parameters,
                                cv=5, scoring='f1_weighted', verbose=1)
        clf.fit(X, y)
        print("best params : {} , best F1: {}".format(clf.best_params_,clf.best_score_))
        f = open(MODEL_PATH, 'wb')
        pickle.dump(clf, f)
        f.close()
        return clf

    @classmethod
    def get_classifier(self):
        '''
            get_classifier function is used to load the classifier.
        '''

        if(self.classifier is None):
            print("Loading classifier")
            try:
                f = open(MODEL_PATH, 'rb')   # 'rb' for reading binary file
                self.classifier = pickle.load(f)
                print("Loaded classifier")
                f.close()
            except:
                print("Exception in loading classifier")
        return self.classifier

    @classmethod
    def predict(self,query):
        '''
            predict function is used to take sentence as input and returns the intents.
        '''

        model = self.get_classifier()
        if(model is not None):
            query_lemmatized = DataHelper.lemmatize(str(query))
            prediction = model.predict_proba(np.array(spacy_model(query_lemmatized).vector).reshape(1,96))
            index_to_label_dict = DataHelper.get_index_label_dict()
            label_index = np.argmax(prediction)
            label = index_to_label_dict[str(label_index)]
            class_score = prediction[0][label_index]
            return {"label":label,
                    "probability":class_score}
        else:
            print("Unable to load classifier")

In [ ]:
# Training the model
IntentClassifier.train()

Fitting 5 folds for each of 6 candidates, totalling 30 fits
best params : {'C': 1, 'kernel': 'linear'} , best F1: 0.8655177800304864


GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', probability=True),
             param_grid=[{'C': [1, 2, 5, 10, 20, 100], 'kernel': ['linear']}],
             scoring='f1_weighted', verbose=1)

In [ ]:
# Example for intent classification
print(IntentClassifier.predict("Who is alan turing")['label'])

Loading classifier
Loaded classifier
who


# Named Entity Recognition

In [ ]:
class NER:
    '''
        NER class is used to get the named entities for given input sentences.
    '''

    @classmethod
    def parts_of_speech(self,message,query_properties):
        '''
            parts_of_speech function is used to identify parts of speech and query
            properies for given input message.
        '''
        
        sentences = nltk.sent_tokenize(message)
        tokenized = [nltk.word_tokenize(sentence) for sentence in sentences]
        pos_tags = [nltk.pos_tag(sentence) for sentence in tokenized]
        print('pos tags:',pos_tags[0])

        for name, tag in pos_tags[0]:
            if tag == 'NN':
                name = name.split("-")
                name = " ".join(name)
                query_properties.append(name)
        print('query_proprties:',query_properties)
        chunked_sents = nltk.ne_chunk_sents(pos_tags, binary=True)

        return chunked_sents

    @classmethod
    def traverse(self,tree):
        '''
            traverse function is used to traverse over the tree to find entities.
        '''
        
        entity_names=[]
        if hasattr(tree, 'label') and tree.label:
            if tree.label()=='NE':
                entity_names.append(' '.join([child[0] for child in tree]))
            else:
                for child in tree:
                    entity_names.extend(self.traverse(child))
        
        return entity_names

    @classmethod
    def named_entities(self,message):
        '''
            named_entities function uses parts_of_speech and traverse methods and identifies
            named entities.
        '''
        
        # Identifying Parts of Speech
        query_properties=[]
        chunks=self.parts_of_speech(message,query_properties)

        entities=[]
        for chunk in chunks:
            temp=sorted(list(set([word for tree in chunk for word in self.traverse(tree)])))
            for e in temp:
                if e not in entities:
                    entities.append(e)
        
        return entities,query_properties

In [ ]:
# Example for named entity recognition
NER.named_entities('Who invented Turing machine')

pos tags: [('Who', 'WP'), ('invented', 'VBD'), ('Turing', 'NNP'), ('machine', 'NN')]
query_proprties: ['machine']


(['Turing'], ['machine'])

# Keyword Extraction

In [ ]:
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

In [ ]:
class KE:
    '''
        KE class is used to identify main keywords in given input sentences.
    '''

    @classmethod
    def lemmatize_words(self,words):
        '''
            lemmatize_words function is used to lemmatize the given input sentence.
        '''
        
        lemmatized_words = []
        for word in words:
            lemmatized_words.append(wordlemmatizer.lemmatize(word))
        
        return lemmatized_words

    @classmethod
    def keywords(self,text):
        '''
            keywords function is used to extract main keywords from given input sentence.
        '''
        
        tokenized_words_with_stopwords = word_tokenize(text)
        tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
        tokenized_words = [word for word in tokenized_words if len(word) > 1]
        tokenized_words = [word.lower() for word in tokenized_words]
        tokenized_words = self.lemmatize_words(tokenized_words)
        return tokenized_words

In [ ]:
# Example for keyword extraction
KE.keywords("Who is Alan Turing")

['who', 'alan', 'turing']

# SPARQL Query Generation

In [ ]:
from SPARQLWrapper import SPARQLWrapper as sq, JSON
from vocabulary.vocabulary import Vocabulary as vb

In [ ]:
class SPARQL:
    '''
        SPARQL class is used to take intents, named entities, keywords and query properties as
        input and based on them it creates two sparql quries for dbpedia, wikidata and execute
        them to retrive response from respective sites.
    '''

    @classmethod
    def question_to_sparql(self,intents,named_entity,keywords,query_properties):
        # Detecting property codes of wikidata
        property_code = []
        if len(query_properties) != 0:
            properties = open('/content/drive/MyDrive/Colab Notebooks/data/query.json', 'r')
            properties = json.load(properties)
            noun = query_properties[0]
            noun_synonyms = vb.synonym(noun, format="dict")

            for p, prop in list(properties.items()):
                if prop == noun:
                    property_code.append(p)
                    break

            if len(property_code) == 0:
                for p, prop in list(properties.items()):
                    if type(noun_synonyms) != bool:
                        for synonym in noun_synonyms.values():
                            if prop == synonym:
                                property_code.append(p)
                                break
            print('property_code:',property_code)

        if len(named_entity) != 0:
            # Creating SPARQL quries
            if len(property_code) != 0:
                flag = 0
                sparql = sq("https://query.wikidata.org/sparql")
                query = """SELECT ?label ?property ?thumbnail WHERE
                            { 
                            ?entity rdfs:label ?label .
                            ?entity wdt:""" + property_code[0] + """ ?property_id .
                            ?property_id rdfs:label ?property .
                            OPTIONAL { ?property_id wdt:P18 ?thumbnail .}
                            FILTER (STR(?label) = '""" + named_entity[0] + """') .
                            FILTER (LANG(?property) = "en")
                            }"""
                print('query:',query)
            else:
                flag = 1
                sparql = sq("https://dbpedia.org/sparql")
                query = """SELECT ?label ?description ?thumbnail WHERE
                    { 
                    ?entity rdfs:label ?label .
                    ?entity dbo:abstract ?description .
                    ?entity dbo:thumbnail ?thumbnail .
                    FILTER (STR(?label) = '""" + named_entity[0] + """' && LANG(?description) = "en") .
                    }
                    LIMIT 1"""
                print('query:',query)

            # Getting response from dbpedia or wikidata
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            global result
            result = list()
            temp = str()
            try:
                results = sparql.query().convert()
                print('results:',results)
                if flag == 0:
                    for data in results["results"]["bindings"]:
                        result.append(data["property"]["value"])
                        if "thumbnail" in data and data["thumbnail"]["value"] != "":
                            temp = data["thumbnail"]["value"]
                else:
                    for data in results["results"]["bindings"]:
                        result.append(data["description"]["value"])
                        if "thumbnail" in data and data["thumbnail"]["value"] != "":
                            temp = data["thumbnail"]["value"]

                result = list(set(result))
                response = []
                for i in result:
                    print(i)
                    response.append(''.join(i).encode('utf-8'))
            except:
                response = ["Unable to retrieve data"]
        else:
            response = ["Unable to retrieve data"]

        print('response:',response)
        if response!=["Unable to retrieve data"] and response!=[]:
          response[0]=response[0].decode('utf-8')
        return response

# Text Summerization

In [ ]:
class TFIDF:
    '''
        TFIDF class is used to summerize the given input message using tf-idf.
    '''

    @classmethod
    def remove_special_characters(self,text):
        '''
            remove_special_characters function is used remove special characters
            from given input sentence.
        '''

        regex = r'[^a-zA-Z0-9\s]'
        text = re.sub(regex,'',text)

        return text

    @classmethod
    def freq(self,words):
        '''
            freq function is used to count the word frequency.
        '''

        words = [word.lower() for word in words]
        dict_freq = {}
        words_unique = []
        for word in words:
            if word not in words_unique:
                words_unique.append(word)
        for word in words_unique:
            dict_freq[word] = words.count(word)

        return dict_freq
    
    @classmethod
    def lemmatize_words(self,words):
        '''
            lemmatize_words function is used to lemmatize the given input sentence.
        '''

        lemmatized_words = []
        for word in words:
            lemmatized_words.append(wordlemmatizer.lemmatize(word))
        
        return lemmatized_words

    @classmethod
    def pos_tagging(self,text):
        '''
            pos_tagging function is used to identify parts of speech for given input sentence.
        '''

        pos_tag = nltk.pos_tag(text.split())
        pos_tagged_noun_verb = []
        for word,tag in pos_tag:
            if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
                pos_tagged_noun_verb.append(word)
        
        return pos_tagged_noun_verb

    @classmethod
    def tf_score(self,word,sentence):
        '''
            tf_score function is used to calculate text frequency score.
        '''

        word_frequency_in_sentence = 0
        len_sentence = len(sentence)
        for word_in_sentence in sentence.split():
            if word == word_in_sentence:
                word_frequency_in_sentence = word_frequency_in_sentence + 1
        tf =  word_frequency_in_sentence/ len_sentence
        
        return tf

    @classmethod
    def idf_score(self,no_of_sentences,word,sentences):
        '''
            idf_score function is used to calculate inverse document frequency score.
        '''

        no_of_sentence_containing_word = 0
        for sentence in sentences:
            sentence = self.remove_special_characters(str(sentence))
            sentence = re.sub(r'\d+', '', sentence)
            sentence = sentence.split()
            sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
            sentence = [word.lower() for word in sentence]
            sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
            if word in sentence:
                no_of_sentence_containing_word = no_of_sentence_containing_word + 1
        idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
        
        return idf

    @classmethod
    def tf_idf_score(self,tf,idf):
        return tf*idf

    @classmethod
    def word_tfidf(self,word,sentences,sentence):
        tf = self.tf_score(word,sentence)
        idf = self.idf_score(len(sentences),word,sentences)
        tf_idf = self.tf_idf_score(tf,idf)
        
        return tf_idf

    @classmethod
    def sentence_importance(self,sentence,dict_freq,sentences):
        '''
            sentence_importance function is used to scores for each sentence.
        '''

        sentence_score = 0
        sentence = self.remove_special_characters(str(sentence)) 
        sentence = re.sub(r'\d+', '', sentence)
        pos_tagged_sentence = self.pos_tagging(sentence)
        for word in pos_tagged_sentence:
            if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
                    word = word.lower()
                    word = wordlemmatizer.lemmatize(word)
                    sentence_score = sentence_score + self.word_tfidf(word,sentences,sentence)
        
        return sentence_score

    @classmethod
    def tf_idf(self,text):
        '''
            tf_idf function is uses above methods by taking given input text and
            returning summarized text.
        '''

        print('text:',text)
        tokenized_sentence = sent_tokenize(text)
        if len(tokenized_sentence)<=4:
            return text
        
        tokenized_words_with_stopwords = word_tokenize(text)
        tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
        tokenized_words = [word for word in tokenized_words if len(word) > 1]
        tokenized_words = [word.lower() for word in tokenized_words]
        tokenized_words = self.lemmatize_words(tokenized_words)
        word_freq = self.freq(tokenized_words)
        no_of_sentences = 2
        c = 1
        sentence_with_importance = {}
        for sent in tokenized_sentence:
            sentenceimp = self.sentence_importance(sent,word_freq,tokenized_sentence)
            sentence_with_importance[c] = sentenceimp
            c = c+1
        sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)
        cnt = 0
        summary = []
        sentence_no = []
        for word_prob in sentence_with_importance:
            if cnt < no_of_sentences:
                sentence_no.append(word_prob[0])
                cnt = cnt+1
            else:
                break
        sentence_no.sort()
        cnt = 1
        for i in range(2):
            summary.append(tokenized_sentence[i])
        for sentence in tokenized_sentence:
            if cnt in sentence_no and sentence not in summary:
                summary.append(sentence)
            cnt = cnt+1
        summary = " ".join(summary)
        
        return str(summary)

In [ ]:
text = "Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist. Turing was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer science and artificial intelligence. Born in Maida Vale, London, Turing was raised in southern England. He graduated at King's College, Cambridge, with a degree in mathematics. Whilst he was a fellow at Cambridge, he published a proof demonstrating that some purely mathematical yes–no questions can never be answered by computation and defined a Turing machine, and went on to prove the halting problem for Turing machines is undecidable. In 1938, he obtained his PhD from the Department of Mathematics at Princeton University. During the Second World War, Turing worked for the Government Code and Cypher School (GC&CS) at Bletchley Park, Britain's codebreaking centre that produced Ultra intelligence. For a time he led Hut 8, the section that was responsible for German naval cryptanalysis. Here, he devised a number of techniques for speeding the breaking of German ciphers, including improvements to the pre-war Polish bombe method, an electromechanical machine that could find settings for the Enigma machine. Turing played a crucial role in cracking intercepted coded messages that enabled the Allies to defeat the Axis powers in many crucial engagements, including the Battle of the Atlantic."

In [ ]:
# Example for text summerization
TFIDF.tf_idf(text)

text: Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist. Turing was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer science and artificial intelligence. Born in Maida Vale, London, Turing was raised in southern England. He graduated at King's College, Cambridge, with a degree in mathematics. Whilst he was a fellow at Cambridge, he published a proof demonstrating that some purely mathematical yes–no questions can never be answered by computation and defined a Turing machine, and went on to prove the halting problem for Turing machines is undecidable. In 1938, he obtained his PhD from the Department of Mathemat

'Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist. Turing was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Whilst he was a fellow at Cambridge, he published a proof demonstrating that some purely mathematical yes–no questions can never be answered by computation and defined a Turing machine, and went on to prove the halting problem for Turing machines is undecidable. For a time he led Hut 8, the section that was responsible for German naval cryptanalysis.'

# Chatterbot

In [ ]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

In [ ]:
class Chatbot:
    # Creation of Bot
    bot = ChatBot(
        'Lucy',
        preprocessors=[
            'chatterbot.preprocessors.clean_whitespace'
        ],
        storage_adapter='chatterbot.storage.SQLStorageAdapter',
        logic_adapters=[
            {
                'import_path': 'chatterbot.logic.BestMatch',
                'default_response': 'I am sorry, but I do not understand.',
                'maximum_similarity_threshold': 1.0
            }
        ],
        database_uri='sqlite:///database.sqlite3',
    )

    # Training
    trainer = ChatterBotCorpusTrainer(bot)
    trainer.train(
        "chatterbot.corpus.english"
    )

    @classmethod
    def chatbot_response(self,message):
        response = self.bot.get_response(message)
        return str(response)

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


# KBot

In [ ]:
class KBOT:
    @classmethod
    def kbot_response(self,message):
        # Intent Classification
        intents=IntentClassifier.predict(message)['label']
        print('Intents:',intents)

        # Named Entity Recognition
        entities,query_properties=NER.named_entities(message)
        print('Entities:',entities)


        # Keyword Extraction
        keywords=KE.keywords(message)
        print('Keywords:',keywords)

        # SPARQL Query Generation
        response=SPARQL.question_to_sparql(intents,entities,keywords,query_properties)
        print('Response after SPARQL execution:',response)

        # Text Summerization
        if response!=['Unable to retrieve data'] and response!=[]:
            response=TFIDF.tf_idf(response[0])
        else:
            response='Unable to retrieve data'
        print('Response after TF-IDF:',response)

        return response

# Examples for chatbot

In [ ]:
KBOT.kbot_response("who is Adolf Hitler")

Intents: who
pos tags: [('who', 'WP'), ('is', 'VBZ'), ('Adolf', 'NNP'), ('Hitler', 'NNP')]
query_proprties: []
Entities: ['Adolf Hitler']
Keywords: ['adolf', 'hitler']
query: SELECT ?label ?description ?thumbnail WHERE
                    { 
                    ?entity rdfs:label ?label .
                    ?entity dbo:abstract ?description .
                    ?entity dbo:thumbnail ?thumbnail .
                    FILTER (STR(?label) = 'Adolf Hitler' && LANG(?description) = "en") .
                    }
                    LIMIT 1
results: {'head': {'link': [], 'vars': ['label', 'description', 'thumbnail']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'label': {'type': 'literal', 'xml:lang': 'ga', 'value': 'Adolf Hitler'}, 'description': {'type': 'literal', 'xml:lang': 'en', 'value': 'Adolf Hitler (German: [ˈadɔlf ˈhɪtlɐ]; 20 April 1889 – 30 April 1945) was an Austrian-born German politician who was the dictator of Germany from 1933 until his death in 1945. He rose

'Adolf Hitler (German: [ˈadɔlf ˈhɪtlɐ]; 20 April 1889 – 30 April 1945) was an Austrian-born German politician who was the dictator of Germany from 1933 until his death in 1945. He rose to power as the leader of the Nazi Party, becoming the chancellor in 1933 and then assuming the title of Führer und Reichskanzler in 1934. Former chancellor Franz von Papen and other conservative leaders persuaded President Paul von Hindenburg to appoint Hitler as chancellor on 30 January 1933. Hitler sought Lebensraum (lit.'

In [ ]:
KBOT.kbot_response("Who is Alan Turing")

Intents: who
pos tags: [('Who', 'WP'), ('is', 'VBZ'), ('Alan', 'NNP'), ('Turing', 'NNP')]
query_proprties: []
Entities: ['Alan Turing']
Keywords: ['who', 'alan', 'turing']
query: SELECT ?label ?description ?thumbnail WHERE
                    { 
                    ?entity rdfs:label ?label .
                    ?entity dbo:abstract ?description .
                    ?entity dbo:thumbnail ?thumbnail .
                    FILTER (STR(?label) = 'Alan Turing' && LANG(?description) = "en") .
                    }
                    LIMIT 1
results: {'head': {'link': [], 'vars': ['label', 'description', 'thumbnail']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'label': {'type': 'literal', 'xml:lang': 'ga', 'value': 'Alan Turing'}, 'description': {'type': 'literal', 'xml:lang': 'en', 'value': 'Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologis

'Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist. Turing was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. He accepted chemical castration treatment, with DES, as an alternative to prison. A 2019 BBC series, as voted by the audience, named him the greatest person of the 20th century.'

In [ ]:
KBOT.kbot_response("Who is Elon Musk")

Intents: who
pos tags: [('Who', 'WP'), ('is', 'VBZ'), ('Elon', 'NNP'), ('Musk', 'NNP')]
query_proprties: []
Entities: ['Elon Musk']
Keywords: ['who', 'elon', 'musk']
query: SELECT ?label ?description ?thumbnail WHERE
                    { 
                    ?entity rdfs:label ?label .
                    ?entity dbo:abstract ?description .
                    ?entity dbo:thumbnail ?thumbnail .
                    FILTER (STR(?label) = 'Elon Musk' && LANG(?description) = "en") .
                    }
                    LIMIT 1
results: {'head': {'link': [], 'vars': ['label', 'description', 'thumbnail']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'label': {'type': 'literal', 'xml:lang': 'fr', 'value': 'Elon Musk'}, 'description': {'type': 'literal', 'xml:lang': 'en', 'value': "Elon Reeve Musk FRS (/ˈiːlɒn/; born June 28, 1971) is an entrepreneur and business magnate. He is the founder, CEO and Chief Engineer at SpaceX; early-stage investor, CEO and Product Architec

'Elon Reeve Musk FRS (/ˈiːlɒn/; born June 28, 1971) is an entrepreneur and business magnate. He is the founder, CEO and Chief Engineer at SpaceX; early-stage investor, CEO and Product Architect of Tesla, Inc.; founder of The Boring Company; and co-founder of Neuralink and OpenAI. Musk has proposed the Hyperloop, a high-speed vactrain transportation system. In 2018, he was sued for defamation by a British caver who advised in the Tham Luang cave rescue; a California jury ruled in favor of Musk.'

# Templates Creation

In [ ]:
import os

os.mkdir("templates")
os.mkdir("static")
os.mkdir("static/css")
os.mkdir("static/js")

with open("/content/drive/MyDrive/Colab Notebooks/templates/index.html",'r') as f1, open("templates/index.html",'w') as f2:
  for line in f1:
    f2.write(line)
f1.close()
f2.close()

with open("/content/drive/MyDrive/Colab Notebooks/static/css/style.css",'r') as f1, open("static/css/style.css",'w') as f2:
  for line in f1:
    f2.write(line)
f1.close()
f2.close()

with open("/content/drive/MyDrive/Colab Notebooks/static/js/app.js",'r') as f1, open("static/js/app.js",'w') as f2:
  for line in f1:
    f2.write(line)
f1.close()
f2.close()

# Flask App Creation

In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

# Flask app creation
app = Flask(__name__)
run_with_ngrok(app)

# Routing to Views

In [ ]:
# Routes
@app.route("/")
def index():
    return render_template("index.html")

@app.route('/get')
def get_bot_response():
    # Getting user's question
    userText = request.args.get('msg')
    if userText=="":
        return "Please ask a question." 
    if userText=='What can you do.' or userText=='What can you do?' or userText=='what can you do.' or userText=='what can you do?' or userText=='What can you do' or userText=='what can you do':
        return "You can chat with me, ask questions and I can bring you answers from Linked Data."
    
    # Getting response from chatterbot
    response=Chatbot.chatbot_response(str(userText))
    if response!='I am sorry, but I do not understand.':
        return response
    
    # Getting response from knowledge bases
    response=KBOT.kbot_response(str(userText))
    if response=='Unable to retrieve data':
        return 'I am sorry, but I do not understand.'
    
    return response

# Running Flask App

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://90c1-35-224-95-180.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
